# D4 DATA Economy Index: Index Weight Calculation

This Notebook calculates the weights for each token within the [DATA Economy Index (DATA)](https://indexcoop.com/data-economy-index) at a given point in time.

The DATA index is maintained quarterly in two phases:

<b>1. Determination Phase:</b>
During the determination phase, the tokens being added and deleted from the index calculation are determined during the final week of the quarter and published before quarterly rebalancing.

<b>2. Rebalancing Phase:</b>
Following publication of the determination phase outcome, the index composition will change to the new weights during the first week of the following quarter.

This Notebook assists with the Determination Phase to maintain the DATA index.

<b>View the Full Methodology:</b> [d4.xyz/data-token](d4.xyz/data-token)

____________________________

### Import Libraries

In [1]:
import json
import requests
import pandas as pd
import os
import numpy as np

from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

### Create List of Data Economy Token Ids

In [2]:
data_economy_token_ids = ['chainlink', 'filecoin', 'the-graph', 'basic-attention-token', 'livepeer', 'ethereum-name-service', 'ocean-protocol', 'numeraire'] 

# Under Considation:
# Tier 1: 'helium', 'arweave', 'handshake'
# Tier 2: 'streamr-xdata', 'fetch-ai', 'pocket-network', 'siacoin', 'storj', 'akash-network',
# Tier 3: 'theta-token', 'render-token',  'audius', 'ankr', 'radicle', 'gitcoin', 'bittorrent'

In [3]:
data_economy_token_ids

['chainlink',
 'filecoin',
 'the-graph',
 'basic-attention-token',
 'livepeer',
 'ethereum-name-service',
 'ocean-protocol',
 'numeraire']

### Pull Data Economy Token Data from CoinGecko API

In [4]:
data_economy_df = pd.DataFrame(cg.get_coins_markets(vs_currency=['usd']
                               , ids=data_economy_token_ids)
                               , columns = ['id', 'symbol', 'market_cap', 'fully_diluted_valuation'
                                           , 'total_volume', 'circulating_supply'
                                           , 'total_supply', 'max_supply'])

In [5]:
data_economy_df

,id,symbol,market_cap,fully_diluted_valuation,total_volume,circulating_supply,total_supply,max_supply
0,chainlink,link,6622292413,1.418021e+10,494309434,4.670095e+08,1.000000e+09,1.000000e+09
1,filecoin,fil,3688784949,3.825378e+10,289313817,1.899901e+08,1.970254e+09,1.970254e+09
2,the-graph,grt,2567051045,3.832980e+09,174543454,6.697272e+09,1.000000e+10,1.000000e+10
3,basic-attention-token,bat,1108512497,1.109101e+09,57662238,1.499205e+09,1.500000e+09,1.500000e+09
4,livepeer,lpt,610313129,6.103131e+08,10332857,2.491851e+07,2.491851e+07,2.491851e+07
5,ethereum-name-service,ens,341402196,1.460163e+09,25360519,2.338110e+07,1.000000e+08,1.000000e+08
6,ocean-protocol,ocean,220397174,NaN,14662011,4.340268e+08,6.130991e+08,NaN
7,numeraire,nmr,162785942,2.999322e+08,2593816,5.970167e+06,1.095410e+07,1.100000e+07


In [6]:
data_economy_df

,id,symbol,market_cap,fully_diluted_valuation,total_volume,circulating_supply,total_supply,max_supply
0,chainlink,link,6622292413,1.418021e+10,494309434,4.670095e+08,1.000000e+09,1.000000e+09
1,filecoin,fil,3688784949,3.825378e+10,289313817,1.899901e+08,1.970254e+09,1.970254e+09
2,the-graph,grt,2567051045,3.832980e+09,174543454,6.697272e+09,1.000000e+10,1.000000e+10
3,basic-attention-token,bat,1108512497,1.109101e+09,57662238,1.499205e+09,1.500000e+09,1.500000e+09
4,livepeer,lpt,610313129,6.103131e+08,10332857,2.491851e+07,2.491851e+07,2.491851e+07
5,ethereum-name-service,ens,341402196,1.460163e+09,25360519,2.338110e+07,1.000000e+08,1.000000e+08
6,ocean-protocol,ocean,220397174,NaN,14662011,4.340268e+08,6.130991e+08,NaN
7,numeraire,nmr,162785942,2.999322e+08,2593816,5.970167e+06,1.095410e+07,1.100000e+07


### Calculate Token Weights based on Market Capitalization

In [7]:
# Calculate Market Cap Weights
data_economy_df['market_cap_weight'] = data_economy_df['market_cap'] / data_economy_df['market_cap'].sum()

In [8]:
data_economy_df[['id', 'symbol', 'market_cap_weight']]

,id,symbol,market_cap_weight
0,chainlink,link,0.432221
1,filecoin,fil,0.240758
2,the-graph,grt,0.167545
3,basic-attention-token,bat,0.072350
4,livepeer,lpt,0.039834
5,ethereum-name-service,ens,0.022282
6,ocean-protocol,ocean,0.014385
7,numeraire,nmr,0.010625


### Adjust Market Cap Weights with 25% Cap

In [9]:
def adjust_market_cap_weight(weights, token_weight_cap = 0.25): 
    """ Inputs: 
    
            - weights: List of numeric token weights that sums to 1.
            
            - token_weight_cap: List of numeric columns within data to be rounded. 
                                Default is 0.25.
    
        Output: 
        
            - weights_adj: List of numeric token weights that sums to 1 
                           where no individual number > token_weight_cap.
    
    """
    weights_adj = weights.copy()

    for i in range(0, len(weights_adj)-1):
        token_weight = weights_adj[i]
        token_overflow = 0.0
        
        if token_weight > token_weight_cap:
            token_overflow = token_weight - token_weight_cap
            weights_adj[i] = token_weight_cap

        if token_overflow > 0.0:
            weights_sum = np.sum(weights_adj[(i+1):])
            weights_temp = weights_adj[(i+1):] + (weights_adj[(i+1):] / weights_sum * token_overflow)

            weights_adj[(i+1):] = weights_temp

        else: 
            pass

    return weights_adj

### Calculate Adjusted Weights

In [10]:
data_economy_df['market_cap_weight_adj'] = adjust_market_cap_weight(data_economy_df['market_cap_weight'], token_weight_cap = 0.25)

In [11]:
data_economy_df

,id,symbol,market_cap,fully_diluted_valuation,total_volume,circulating_supply,total_supply,max_supply,market_cap_weight,market_cap_weight_adj
0,chainlink,link,6622292413,1.418021e+10,494309434,4.670095e+08,1.000000e+09,1.000000e+09,0.432221,0.250000
1,filecoin,fil,3688784949,3.825378e+10,289313817,1.899901e+08,1.970254e+09,1.970254e+09,0.240758,0.250000
2,the-graph,grt,2567051045,3.832980e+09,174543454,6.697272e+09,1.000000e+10,1.000000e+10,0.167545,0.250000
3,basic-attention-token,bat,1108512497,1.109101e+09,57662238,1.499205e+09,1.500000e+09,1.500000e+09,0.072350,0.113419
4,livepeer,lpt,610313129,6.103131e+08,10332857,2.491851e+07,2.491851e+07,2.491851e+07,0.039834,0.062445
5,ethereum-name-service,ens,341402196,1.460163e+09,25360519,2.338110e+07,1.000000e+08,1.000000e+08,0.022282,0.034931
6,ocean-protocol,ocean,220397174,NaN,14662011,4.340268e+08,6.130991e+08,NaN,0.014385,0.022550
7,numeraire,nmr,162785942,2.999322e+08,2593816,5.970167e+06,1.095410e+07,1.100000e+07,0.010625,0.016656


### Round Numeric Column Data

In [12]:
def round_columns(data, columns):
    """ Inputs: 
    
            - data: DataFrame representing Data Economy token data.
            
            - columns: list of numeric columns within data to be rounded.
    
        Output: 
        
            - data: DataFrame with numeric columns rounded.
    
    """
    
    for col in columns: 
        data[col] = np.round(data[col], 5)
        
    return data

In [13]:
data_economy_df = round_columns(data_economy_df, columns = ['market_cap_weight', 'market_cap_weight_adj'])

In [14]:
data_economy_df = data_economy_df.sort_values(by = 'market_cap', ascending = False).reset_index(drop=True)

### Save Token Weight DataFrames to CSV

In [15]:
data_economy_df[['id', 'symbol', 'market_cap_weight_adj']].to_csv('Data/final_token_weights.csv')
data_economy_df[['id', 'symbol', 'market_cap_weight_adj']].sort_values(by = 'id', ascending = True).reset_index(drop=True).to_csv('Data/final_token_weights_alphabetical.csv')

### Code to Screen Capture Token Weights

In [16]:
# Recording: Windows + Alt + R

# python
# import pandas as pd
# import os
# data_df = pd.read_csv('final_token_weights.csv', sep = ',')
# data_df